In [1]:
from importlib import reload
import json
import logging
import time
from argparse import ArgumentParser
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

import datasets
from model import CNN, MyDartsTrainer

import utils

c:\Users\pkbab\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
layers = 1
batch_size = 64
log_frequency = 40
channels = 16
unrolled = False
visualization = False

In [3]:
dataset = "cifar100"

dataset_train, dataset_valid = datasets.get_dataset(dataset)        

Files already downloaded and verified
Files already downloaded and verified


# Architecture search for a range of $\lambda$

In [11]:
logs = {}

In [13]:
epochs = 50
lambd = 1
n_chosen = 1
weight = 0.1
wt = None
for number in [1, 2, 3]:
    if dataset == "fashionmnist":
        model = CNN(32, 1, channels, 10, layers, n_chosen=1)
    elif dataset == "cifar10":
        model = CNN(32, 3, channels, 10, layers, n_chosen=1)
    elif dataset == "cifar100":
        model = CNN(32, 3, channels, 100, layers, n_chosen=1)

    criterion = nn.CrossEntropyLoss() # mycriterion()
    optim = torch.optim.SGD(model.parameters(), 0.025, momentum=0.9, weight_decay=3.0E-4)
    lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optim, epochs, eta_min=0.001)
    trainer = MyDartsTrainer(
        model=model,
        loss=criterion, # =mycriterion,
        metrics=lambda output, target: utils.accuracy(output, target, topk=(1,)),
        optimizer=optim,
        num_epochs=epochs,
        dataset=dataset_train,
        batch_size=batch_size,
        log_frequency=log_frequency,
        unrolled=unrolled,
        weight=weight, # вес регуляризатора
        lambd=lambd, # количество общих ребер
        train_as_optimal=True,
        optimalPath='checkpoints/CIFAR100/optimal/arc.json',
        tau=1.0,
        learning_rate=2.5E-3,
        arc_learning_rate=3.0E-1,
        n_chosen=1,
        t_alpha=0.2,
        t_beta=0.2,
    )
    trainer.fit()
    final_architecture = trainer.export()
    print('Final architecture:', final_architecture)
    json.dump(trainer.export(), open(f'checkpoints/CIFAR100/random/{number}/arc.json', 'w+'))

Epoch [1/50] Step [1/391]  acc1 0.015625 (0.015625)  loss 4.607870 (4.607870)
Epoch [1/50] Step [41/391]  acc1 0.031250 (0.020579)  loss 4.443074 (4.575597)
Epoch [1/50] Step [81/391]  acc1 0.046875 (0.027199)  loss 4.323179 (4.490315)
Epoch [1/50] Step [121/391]  acc1 0.031250 (0.037319)  loss 4.183802 (4.405519)
Epoch [1/50] Step [161/391]  acc1 0.046875 (0.042508)  loss 4.205765 (4.353607)
Epoch [1/50] Step [201/391]  acc1 0.109375 (0.048896)  loss 3.841792 (4.297391)
Epoch [1/50] Step [241/391]  acc1 0.093750 (0.054914)  loss 4.039520 (4.254007)
Epoch [1/50] Step [281/391]  acc1 0.093750 (0.058997)  loss 3.999937 (4.225382)
Epoch [1/50] Step [321/391]  acc1 0.093750 (0.062646)  loss 3.935636 (4.198299)
Epoch [1/50] Step [361/391]  acc1 0.062500 (0.065789)  loss 3.965824 (4.169684)
Epoch [2/50] Step [1/391]  acc1 0.093750 (0.093750)  loss 3.886081 (3.886081)
Epoch [2/50] Step [41/391]  acc1 0.125000 (0.117759)  loss 3.647569 (3.896741)
Epoch [2/50] Step [81/391]  acc1 0.125000 (0.11

# Retrain

In [4]:
layers = 2
batch_size = 96
log_frequency = 20
channels = 16
unrolled = False
visualization = False
dataset = 'cifar100'

In [5]:
from retrain import train, validate, fixed_arch
# reload(train)

config = {
'layers' : layers,
'batch_size' : batch_size,
'log_frequency' : log_frequency,
'epochs' : 50,
'aux_weight' : 0.4,
'drop_path_prob' : 0.1,
'workers' : 4,
'grad_clip' : 5.,
'save_folder' : f"./checkpoints/{dataset}/",
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataset_train, dataset_valid = datasets.get_dataset(dataset, cutout_length=16)

best_top1s = {}
for number in [1, 2, 3]:
    folder = config['save_folder'] + f"random/{number}/"
    print(folder)
    with fixed_arch(folder + 'arc.json'):
        if dataset == 'fashionMNIST':
            model = CNN(32, 1, 36, 10, config['layers'], auxiliary=True)
        if dataset == 'cifar100':
            model = CNN(32, 3, 36, 100, config['layers'], auxiliary=True)

    criterion = nn.CrossEntropyLoss()

    model.to(device)
    criterion.to(device)

    optimizer = torch.optim.SGD(model.parameters(), 0.025, momentum=0.9, weight_decay=3.0E-4)
    lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, config['epochs'], eta_min=1E-6)

    train_loader = torch.utils.data.DataLoader(dataset_train,
                                            batch_size=config['batch_size'],
                                            shuffle=True,
                                            num_workers=config['workers'],
                                            pin_memory=True)
    valid_loader = torch.utils.data.DataLoader(dataset_valid,
                                            batch_size=config['batch_size'],
                                            shuffle=False,
                                            num_workers=config['workers'],
                                            pin_memory=True)

    best_top1 = 0.
    for epoch in range(config['epochs']):
        drop_prob = config['drop_path_prob'] * epoch / config['epochs']
        model.drop_path_prob(drop_prob)

        # training
        train(config, train_loader, model, optimizer, criterion, epoch)

        # validation
        cur_step = (epoch + 1) * len(train_loader)
        top1 = validate(config, valid_loader, model, criterion, epoch, cur_step)
        best_top1 = max(best_top1, top1)

        lr_scheduler.step()

    torch.save(model.state_dict(), folder + "mod.json")
    # torch.save(model.state_dict(), args.save_folder + "/mod.json")
    print("Final best Prec@1 = {:.4%}".format(best_top1))
    best_top1s.update({f'number={number}' : best_top1})
print(best_top1s)

Files already downloaded and verified
Files already downloaded and verified
./checkpoints/cifar100/random/1/
[2024-01-15 09:11:58] Fixed architecture: {'reduce_n2_p0': 'sepconv3x3', 'reduce_n2_p1': 'sepconv5x5', 'reduce_n3_p0': 'maxpool', 'reduce_n3_p1': 'sepconv5x5', 'reduce_n3_p2': 'dilconv5x5', 'reduce_n4_p0': 'dilconv3x3', 'reduce_n4_p1': 'sepconv5x5', 'reduce_n4_p2': 'sepconv5x5', 'reduce_n4_p3': 'sepconv5x5', 'reduce_n5_p0': 'dilconv5x5', 'reduce_n5_p1': 'skipconnect', 'reduce_n5_p2': 'dilconv3x3', 'reduce_n5_p3': 'sepconv3x3', 'reduce_n5_p4': 'maxpool', 'reduce_n2_switch': [0], 'reduce_n3_switch': [2], 'reduce_n4_switch': [3], 'reduce_n5_switch': [4]}
[2024-01-15 09:11:58] Epoch 0 LR 0.025000
[2024-01-15 09:12:05] Train: [  1/50] Step 000/520 Loss 6.551 Prec@(1,5) (0.0%, 4.2%)
[2024-01-15 09:12:05] Train: [  1/50] Step 020/520 Loss 6.450 Prec@(1,5) (1.1%, 6.3%)
[2024-01-15 09:12:05] Train: [  1/50] Step 040/520 Loss 6.345 Prec@(1,5) (1.9%, 9.0%)
[2024-01-15 09:12:06] Train: [  1

# Validation

In [15]:
from retrain import train, validate, fixed_arch
import numpy as np
from glob import glob
from nni.retiarii.oneshot.pytorch.utils import AverageMeter
from torch.utils.tensorboard import SummaryWriter

logger = logging.getLogger('nni')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
writer = SummaryWriter()
n_chosen = 1

config = {
'layers' : 2,
'batch_size' : 96,
'log_frequency' : 30,
'epochs' : 10,
'aux_weight' : 0.4,
'drop_path_prob' : 0.1,
'workers' : 4,
'grad_clip' : 5.,
'save_folder' : "./checkpoints/cifar100/random/",
}

dataset_train, dataset_valid = datasets.get_dataset("cifar100", cutout_length=16)

res_dict_accur = {}
models = []

chosen_numbers = [3]

print(chosen_numbers)

for dir in glob(config['save_folder'] + "*"):
    if int(dir.split('\\')[-1]) in chosen_numbers:
        print(dir + "/arc.json")
        with fixed_arch(dir + "/arc.json"):
            model = CNN(32, 3, 36, 100, config['layers'], auxiliary=True, n_chosen=n_chosen)
        model.to(device)
        model.load_state_dict(torch.load(dir + "/mod.json"))
        model.eval()
        
        models.append(model)
         

print(f"Models in ensemble: {len(models)}")

valid_loader = torch.utils.data.DataLoader(dataset_valid,
                                            batch_size=config['batch_size'],
                                            shuffle=False,
                                            num_workers=config['workers'],
                                            pin_memory=True)
criterion = nn.CrossEntropyLoss()

top1 = AverageMeter("top1")
top5 = AverageMeter("top5")
losses = AverageMeter("losses")

# validation
softmax = nn.Softmax(dim=1)
for step, (X, y) in enumerate(valid_loader):
        X, y = X.to(device, non_blocking=True), y.to(device, non_blocking=True)
        bs = X.size(0)

        probabilities = softmax(models[0](X))
        for i in range(1, len(models)):
            probabilities += softmax(models[i](X))
        probabilities = probabilities / len(models)
        loss = criterion(probabilities, y)

        accuracy = utils.accuracy(probabilities, y, topk=(1, 5))
        losses.update(loss.item(), bs)
        top1.update(accuracy["acc1"], bs)
        top5.update(accuracy["acc5"], bs)

        if step % config['log_frequency'] == 0 or step == len(valid_loader) - 1:
            logger.info(
                "Valid: Step {:03d}/{:03d} Loss {losses.avg:.3f} "
                "Prec@(1,5) ({top1.avg:.1%}, {top5.avg:.1%})".format(
                    step, len(valid_loader) - 1, losses=losses,
                    top1=top1, top5=top5))

logger.info("Final best Prec@1 = {:.4%}".format(top1.avg))

# res_dict_accur[chosen_lambdas] = top1.avg
print(top1.avg)

Files already downloaded and verified
Files already downloaded and verified
[3]
./checkpoints/cifar100/random\3/arc.json
[2024-01-15 20:24:24] Fixed architecture: {'reduce_n2_p0': 'sepconv5x5', 'reduce_n2_p1': 'maxpool', 'reduce_n3_p0': 'sepconv5x5', 'reduce_n3_p1': 'dilconv3x3', 'reduce_n3_p2': 'sepconv5x5', 'reduce_n4_p0': 'avgpool', 'reduce_n4_p1': 'dilconv3x3', 'reduce_n4_p2': 'dilconv3x3', 'reduce_n4_p3': 'sepconv5x5', 'reduce_n5_p0': 'dilconv5x5', 'reduce_n5_p1': 'sepconv5x5', 'reduce_n5_p2': 'sepconv3x3', 'reduce_n5_p3': 'sepconv5x5', 'reduce_n5_p4': 'skipconnect', 'reduce_n2_switch': [1], 'reduce_n3_switch': [2], 'reduce_n4_switch': [3], 'reduce_n5_switch': [3]}
Models in ensemble: 1
[2024-01-15 20:24:28] Valid: Step 000/104 Loss 4.025 Prec@(1,5) (64.6%, 86.5%)
[2024-01-15 20:24:28] Valid: Step 030/104 Loss 4.049 Prec@(1,5) (60.8%, 87.1%)
[2024-01-15 20:24:29] Valid: Step 060/104 Loss 4.047 Prec@(1,5) (61.4%, 87.3%)
[2024-01-15 20:24:29] Valid: Step 090/104 Loss 4.047 Prec@(1,5